moving low numbers down to 0 seems to help

remove uninmportant parameters

In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt

from XGBoost_model_testing import model_testing_xgboost
from plot_predictions import plot_predictions
from rmses import rmse_evaluator

Dataframes have the final value removed as this is for the prediction

In [2]:
df = pd.read_pickle('../data/fe_temp_data.pkl')
df = df.iloc[:len(df)-1]
df_cyclic = pd.read_pickle('../data/fe_temp_cyclic_data.pkl')
df_cyclic = df_cyclic.iloc[:len(df_cyclic)-1]


In [3]:
test_dates = [
        '2023-02-12 21:00',
        '2023-02-07 11:00',
        '2023-04-15 08:00',
        '2023-04-15 16:00'
    ]

In [4]:
df_features = ['tempmax', 'tempmin', 'temp', 'precip',
       'snow', 'snowdepth', 'windspeedmean', 'solarenergy', 'year', 'quarter',
       'month', 'day_of_year', 'day_of_month', 'day_of_week', 'hour', 'minute',
       'time', 'minute_of_day', 'sum_3min', 'sum_5min', 'sum_10min',
       'sum_15min', 'sum_20min', 'sum_30min', 'sum_45min', 'sum_1h', 'sum_24h',
       'rolling_mean_30min', 'rolling_std_30min', 'rolling_mean_1h',
       'rolling_std_1h', 'rolling_mean_24h', 'rolling_std_24h',
       'rolling_min_24h', 'rolling_max_24h']

df_cyclic_features = ['tempmax', 'tempmin', 'temp', 'precip',
       'snow', 'snowdepth', 'windspeedmean', 'solarenergy', 'year',
       'sum_3min', 'sum_5min', 'sum_10min', 'sum_15min', 'sum_20min',
       'sum_30min', 'sum_45min', 'sum_1h', 'sum_24h', 'rolling_mean_30min',
       'rolling_std_30min', 'rolling_mean_1h', 'rolling_std_1h',
       'rolling_mean_24h', 'rolling_std_24h', 'rolling_min_24h',
       'rolling_max_24h', 'quarter_sin', 'quarter_cos', 'day_of_month_sin',
       'day_of_month_cos', 'hour_sin', 'hour_cos', 'minute_of_day_sin',
       'minute_of_day_cos', 'minute_sin', 'minute_cos', 'month_sin',
       'month_cos', 'day_of_year_sin', 'day_of_year_cos', 'day_of_week_sin',
       'day_of_week_cos']

In [5]:
X = df[df_features]
y = df['demand']

X_cyclic = df_cyclic[df_cyclic_features]
y_cyclic = df_cyclic['demand']

In [6]:
def train_split(X,y):
    train_size = int(len(X) * 0.8)
    X_train, X_test = X[:train_size], X[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]

    non_numeric_columns = X_train.select_dtypes(include=['object']).columns
    X_train = X_train.drop(columns=non_numeric_columns)
    X_test = X_test.drop(columns=non_numeric_columns)
    return X_train, X_test, y_train, y_test

In [7]:
X_train,X_test,y_train,y_test = train_split(X,y)
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict(X_test)

In [ ]:
importances = xgb_model.feature_importances_
sorted_idx = np.argsort(importances)[::-1]
top_10_idx = sorted_idx[:10]

plt.figure(figsize=(10, 6))
plt.barh(range(len(top_10_idx)), importances[top_10_idx])
plt.yticks(range(len(top_10_idx)), [X_train.columns[i] for i in top_10_idx])
plt.title("Top 10 XGBoost Feature Importance")
plt.gca().invert_yaxis()
plt.show()

In [ ]:
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"mae: {mae}")
print(f"rmse: {rmse}")

plt.figure(figsize=(10, 6))
plt.plot(y_test.values, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.legend()
plt.title('XGBoost Actual vs Predicted')
plt.show()

In [10]:
X_train,X_test,y_train,y_test = train_split(X_cyclic,y_cyclic)
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict(X_test)

In [ ]:
importances = xgb_model.feature_importances_
sorted_idx = np.argsort(importances)[::-1]
top_10_idx = sorted_idx[:10]

plt.figure(figsize=(10, 6))
plt.barh(range(len(top_10_idx)), importances[top_10_idx])
plt.yticks(range(len(top_10_idx)), [X_train.columns[i] for i in top_10_idx])
plt.title("Top 10 XGBoost Feature Importance")
plt.gca().invert_yaxis()
plt.show()

In [ ]:
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f"mae: {mae}")
print(f"rmse: {rmse}")

plt.figure(figsize=(10, 6))
plt.plot(y_test.values, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.legend()
plt.title('XGBoost Actual vs Predicted')
plt.show()